#### Faiss
Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [4]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader('../resources/speech.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30)
docs = text_splitter.split_documents(documents)

In [7]:
docs

[Document(metadata={'source': '../resources/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': '../resources/speech.txt'}, page_content='…\n\nIt will be all the ea

In [13]:
embeddings = OllamaEmbeddings(model="embeddinggemma:latest")
db = FAISS.from_documents(docs, embeddings)
db

In [14]:
### querying
query="How does the speaker describe the desired outcome of the war?"
docs=db.similarity_search(query)
docs

[Document(id='7612b4b2-cfcd-4431-a7e2-394d181a0dbc', metadata={'source': '../resources/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(id='3eb7a6d7-e1e0-4f89-a4d2-3bfee509fffb', me

In [17]:
docs[0].page_content

'The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'

#### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [19]:
retriever = db.as_retriever()
docs = retriever.invoke(query)
docs

[Document(id='7612b4b2-cfcd-4431-a7e2-394d181a0dbc', metadata={'source': '../resources/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(id='3eb7a6d7-e1e0-4f89-a4d2-3bfee509fffb', me

In [20]:
docs[0].page_content

'The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [23]:
docs_and_score = db.similarity_search_with_score(query)
docs_and_score

[(Document(id='7612b4b2-cfcd-4431-a7e2-394d181a0dbc', metadata={'source': '../resources/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
  np.float32(0.940017)),
 (Document(id='3eb7a6d7-e1e0-

In [24]:
embedding_vector = embeddings.embed_query(query)
embedding_vector

[-0.07435819506645203,
 0.0982455462217331,
 0.007814964279532433,
 0.0002269482210977003,
 -0.022645726799964905,
 0.07844466716051102,
 -0.054658859968185425,
 0.003072190098464489,
 -0.004175569396466017,
 -0.06410937756299973,
 0.1016932874917984,
 -0.01607811450958252,
 0.025618206709623337,
 0.019463250413537025,
 0.020425017923116684,
 -0.01740064099431038,
 0.004953558556735516,
 0.013631829060614109,
 -0.04600246250629425,
 -0.006770819891244173,
 -0.030387461185455322,
 -0.058058980852365494,
 0.002799782669171691,
 0.012654222548007965,
 -0.031818754971027374,
 0.0683387741446495,
 0.06195501238107681,
 -0.029265061020851135,
 -0.021182501688599586,
 0.04510152339935303,
 0.014862713403999805,
 0.011023529805243015,
 0.08090775460004807,
 -0.006920104380697012,
 -0.03583751618862152,
 -0.024630535393953323,
 -0.028615308925509453,
 -0.023675376549363136,
 -0.03168769180774689,
 -0.031479835510253906,
 -0.037712011486291885,
 0.04419784992933273,
 -0.024877440184354782,
 -0.0

In [26]:
docs_score = db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='7612b4b2-cfcd-4431-a7e2-394d181a0dbc', metadata={'source': '../resources/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(id='3eb7a6d7-e1e0-4f89-a4d2-3bfee509fffb', me

### Saving and loading

In [27]:
db.save_local("faiss_index")

In [30]:
new_db=FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)

In [31]:
docs

[Document(id='7612b4b2-cfcd-4431-a7e2-394d181a0dbc', metadata={'source': '../resources/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(id='3eb7a6d7-e1e0-4f89-a4d2-3bfee509fffb', me

In [32]:
docs[0].page_content

'The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'